# 使用gymnasium作为智能体的模拟环境

过去两三年，gym，一个openai开发的仓库比较流行，但是到2023年openai放弃了对它的维护。

Farama基金会，一个也是做强化学习库的中立组织，接手了这个工作，并将其更名为gymnasium。

致力于接入更多的强化学习算法、原语和加速。

相关说明参考该[链接](https://farama.org/Announcing-The-Farama-Foundation)。

## Hello World

文档参考该[链接](https://gymnasium.farama.org/content/basic_usage/)

习惯：

```
import gymasium as gym
```

强化学习中的概念[s, a, r, p, \gamma]，状态、动作、奖励、转移概率、衰减系数和gymasium中api的对应关系：

状态空间是env.observation_space，动作空间是env.action_space。实现上从Space类继承。

状态空间和动作空间有很多预定义的空间类型（ndarray类型）：

- Box：描述了一个 n 维连续空间。这是一个有界空间，我们可以定义上限和下限，这些限制描述了我们的观察可以采用的有效值。

- Discrete：描述了一个离散空间，其中 {0, 1, …, n-1} 是观察或动作可能采用的可能值。可以使用可选参数将值移位为 {a, a+1, …, a+n-1}。

- Dict：表示简单空间的字典。

- Tuple：表示简单空间的元组。

- MultiBinary：创建一个 n 维的二进制空间。参数 n 可以是一个数字或数字列表。

- MultiDiscrete：由一系列不同动作数的离散动作空间组成，每个元素中的动作数都不同。

还有一些更专业的空间类型，如 Graph、Sequence 和 Text。[文档1](https://gymnasium.farama.org/api/spaces/)描述了不同空间的使用示例。[文档2](https://gymnasium.farama.org/api/spaces/utils/)描述了一些有用的函数。

In [1]:
import gymnasium as gym

print(gym.__version__) ## 查看版本

env = gym.make("CartPole-v1") ## 初始化环境用make函数，返回一个Env对象。

# print(gym.envs.registry.keys()) ## 查看能够创建的环境列表

## render_mode表示用什么交互界面展示环境
# env = gym.make("LunarLander-v2", render_mode="human")

# Env的reset函数表示重置环境，返回状态空间观测值和info。
observation, info = env.reset(seed=42)
print(env.observation_space)
print(env.observation_space.shape[0])

for _ in range(1000):
    action = env.action_space.sample()

    # step表示智能体与环境交互，返回五元组：状态空间观测值、奖励、是否到达终止态、智能体是否越界使得交互提前终止、info。
    observation, reward, terminated, truncated, info = env.step(action)

    if terminated or truncated:
        observation, info = env.reset()
env.close()

0.28.1
Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)
4


## 一些简单的环境

Cartpole：倒立摆

动作空间：Discrete(2) 离散动作空间，0向左，1向右，

观测空间：Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32) （ndarray描述：4维float32类型），第一维表示位置信息，第二维表示速度，第三维表示倾角，第四维表示角速度。

奖励：只要向右，则+1

初始位置：（-0.05，0.05）之间随机

Episode结束条件：

- termination：倾角大于正负12°，或者距离超过正负2.4，

- truncation：Episode长度大于500

转移概率：**p(s',r|s,a)未知**。

### 一些注释

model-free和model-based：转移概率已知则是model-based，动态规划和蒙特卡洛就是model-based的，因为转移函数已知。蒙特卡洛针对样本空间巨大的问题（围棋），采样部分数据进行训练。动态规划则记录所有状态的值（背包问题）。

value-based和policy-based：智能体学习环境中不同状态的值，根据状态值隐式确定策略就是value-based的方法，直接学习策略的值是policy-based，需要用到策略梯度定理证明策略梯度下降方向。两者结合是actor-acritic方法。

Monte-Carlo update和Temporal-Difference Update：回合更新和单步更新

On-policy和Off-policy：On-policy使用最新的策略，off-policy可以用经验回放。